<a href="https://colab.research.google.com/github/mgontarz20/mgontarz20/blob/main/GC_KFoldValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Imports


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/BadaniaMchtr')
import numpy as np
import os
import tqdm
from tqdm import tqdm
from keras.losses import MeanSquaredError, SparseCategoricalCrossentropy
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import img_to_array, load_img
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard,CSVLogger,LambdaCallback
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from sklearn.model_selection import KFold
import json


#User Imports

In [ ]:
import genConfig
import plotter
import UNetResNet_5lvl

# Definition of hyperparameters

In [ ]:
test_size = 0.25
random_state = 10
n_filters = 4
stop_patience = 14
lr_reducing_patience = 4
batch_size = 16
epoch_limit = 200
norm = False
coeff = 1.0
if norm:
    coeff = 255.0
activation_function = 'relu'
num_folds = 5

#Name definition



In [ ]:
date = datetime.now().strftime("%m-%d-%Y_%H-%M-%S")
root = '/content/drive/MyDrive/BadaniaMchtr/'
pathtoDataSet = f"{root}Datasets/"
dataset = "dataset_9_Combined_to_pred_256x256_10-11-2021_11-24-53"
type1 = "resc_wrpd"
fname_input = "X.npy"
fname_output = "y.npy"
type2 = "resc"
comment = f"SSIMloss_{num_folds}Fold_Validation"
name = f"UNetResNet5lvl_{type1}_{date}_{comment}"

# Path Definition
# Folder Generation

In [ ]:
os.makedirs(f'{root}Results/{name}', exist_ok=True)
os.makedirs(f'{root}Results/{name}/model', exist_ok=True)
os.makedirs(f'{root}Results/{name}/cfg', exist_ok=True)
cfg_dir = f'{root}Results/{name}/cfg'

In [ ]:
path_to_inputs = os.path.join(pathtoDataSet,dataset,fname_input).replace("\\", "/")
path_to_outputs = os.path.join(pathtoDataSet,dataset,fname_output).replace("\\", "/")
print(f"Input path: {path_to_inputs}")
print(f"Output path: {path_to_outputs}")

Input path: /content/drive/MyDrive/BadaniaMchtr/Datasets/dataset_9_Combined_to_pred_256x256_10-11-2021_11-24-53/X.npy
Output path: /content/drive/MyDrive/BadaniaMchtr/Datasets/dataset_9_Combined_to_pred_256x256_10-11-2021_11-24-53/y.npy


# Custom Functions (Callbacks, Loss, KFold)

In [ ]:
json_log = open(f'{root}Results/{name}/{name}_loss_log.json', mode='wt', buffering=1)
json_logging_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: json_log.write(
        json.dumps({'epoch': epoch, 'loss': logs['loss']}) + '\n'),
    on_train_end=lambda logs: json_log.close())

In [ ]:
def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 255.0))

In [ ]:
kfold = KFold(n_splits=num_folds, shuffle=True)


In [ ]:
callbacks = [
    EarlyStopping(patience=stop_patience, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=lr_reducing_patience, min_lr=0.0000001, verbose=1),
    ModelCheckpoint(f'{root}Results/{name}/model/{name}.h5', verbose=1, save_best_only=True),
    #Tensordash(ModelName=f"{name}", email='mgontarz15@gmail.com', password='dupadupa'),
    TensorBoard(log_dir=f"{root}Results/{name}/model/logs", write_graph=True, write_images= True, update_freq=5),
    CSVLogger(f"{root}Results/{name}/model/{name}.csv"),
    json_logging_callback,
            ]

# Importing Images

In [ ]:
X = np.load(path_to_inputs)
y = np.load(path_to_outputs)

print(X.shape, y.shape)

(5001, 256, 256, 1) (5001, 256, 256, 1)


In [ ]:
genConfig.write_cfg(cfg_dir, name, 'w', type_input = type1, type_output= type2, dataset = dataset, num_folds = num_folds)

# Train Test Splitting


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)
del X
del y

In [ ]:
genConfig.write_cfg(cfg_dir, name, "a", test_size = test_size, random_state = random_state)

#Model Loading and Initialization

In [ ]:
for train, test in kfold.split(X, y):

    input_img = Input((256, 256, 1), name='img')
    model = UNetResNet_5lvl.get_unet(input_img, n_filters=n_filters, activation=activation_function)

    model.compile(optimizer=Adam(learning_rate=0.01), loss=SSIMLoss,  metrics=["accuracy"])
    #model.summary()



    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    results = model.fit(X[train], y[train], batch_size=batch_size, epochs=epoch_limit, callbacks=callbacks,
                    validation_data=(X[test], y[test]))
    scores = model.evaluate(X[test], y[test], verbose=0)
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no = fold_no + 1

In [ ]:
genConfig.write_cfg(cfg_dir,name, 'a', optimizer = "Adam", loss = comment, metrics = "accuracy", state = "new", n_filters = n_filters)

#Model Fitting

In [ ]:
time = stop - start

In [ ]:
genConfig.write_cfg(cfg_dir,name, "a", batch_size = batch_size, epoch_limit = epoch_limit, time = time, activation_function = activation_function)


In [ ]:
plotter.plot_model_data(model, name, results, root)
genConfig.write_cfg(cfg_dir,name, "a",  best_loss = np.min(results.history["val_loss"]), for_epoch_loss = np.argmin(results.history["val_loss"]), best_acc = np.max(results.history["val_accuracy"]), for_epoch_acc = np.argmax(results.history["val_accuracy"]))


#K-Fold Validation Scores

In [ ]:
summary = '\n------------------------------------------------------------------------\nScore per fold'
genConfig.write_cfg(cfg_dir,name, "a", summary = summary)
for i in range(0, len(acc_per_fold)):
    per_fold = f"\n------------------------------------------------------------------------\n'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%"
    genConfig.write_cfg(cfg_dir, name, "a", per_fold = per_fold)

In [ ]:
avg_across_all = f'\n------------------------------------------------------------------------\nAverage scores for all folds:\n> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)}) > Loss: {np.mean(loss_per_fold)})'
genConfig.write_cfg(cfg_dir,name, "a", avg_accross_all = avg_across_all)